# Introduction
In this notebook I implement assistant, that answers user question based on context, retrieved from search engine

## Concept scheme

In [4]:
def llm(prompt) -> str:
    return ""

def search(query: str) -> str:
    return ""

def build_prompt(query: str, search_results: str) -> str:
    return ""

def rag(query: str) -> str:
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

## Read data, prepare utilites

In [24]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-08 06:26:58--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0.02s   

2024-09-08 06:26:59 (173 KB/s) - ‘minsearch.py’ saved [3832/3832]



In [27]:
import elasticsearch
from anthropic import Anthropic
import json
import minsearch

In [8]:
with open('../data/site_content.json', 'r') as f_in:
    raw_doc = json.load(f_in)

In [28]:
clientA = Anthropic()

In [30]:
q = "How to get vise to New Zealand"
response = clientA.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens = 500,
    messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": q
            }
          ]
        }
  ]
)
print(response.content[0].text)

Here are the steps to get a visa to travel to New Zealand:

1. Determine the type of visa you need:
   - Visitor visa - for tourism, business, or visiting family/friends
   - Work visa - if you have a job offer in New Zealand
   - Student visa - if you'll be studying in New Zealand

2. Check the specific requirements for the visa you need on the Immigration New Zealand website (https://www.immigration.govt.nz/new-zealand-visas). This will include things like:
   - Application form
   - Valid passport
   - Supporting documents (proof of funds, return ticket, etc.)
   - Application fee

3. Gather all the required documents and information.

4. Complete the online visa application form on the Immigration New Zealand website.

5. Pay the application fee online.

6. Submit any required supporting documents. This may be done online or by post, depending on the visa type.

7. Wait for a decision on your visa application. Processing times can vary, so check the current processing times on the 

In [35]:
data = [{'url': k, 'header':v['header'], 'main_content':v['main_content']} for k,v in raw_doc.items()]

In [36]:
index = minsearch.Index(
    text_fields=["url", "header", "main_content"],
    keyword_fields=[]
)

In [39]:
index.fit(data)

# Basic implementation

In [40]:
def search(query: str) -> str:

    boost = {'header': 2.0, 'url': 5}
    results = index.search(
        query=query,
        boost_dict=boost,
        num_results=5
    )

    return results

In [41]:
q = "How to get student's visa"

In [42]:
search(q)

[{'url': 'https://www.immigration.govt.nz/new-zealand-visas/preparing-a-visa-application/the-application-process/how-to-get-a-temporary-visa-decision-reviewed-1',
  'header': 'Home New Zealand visas Preparing a visa application The application process How to get a temporary visa decision reviewed How to get a temporary visa decision reviewed If we decline your application for a temporary visa to visit, work or study in New Zealand in some cases you can ask us to reconsider our decision. Cost NZD $250 Processing time 80% within 6 weeks',
  'main_content': ''},
 {'url': 'https://www.immigration.govt.nz/new-zealand-visas/visas/visa/pathway-student-visa',
  'header': "Home New Zealand visas Pathway Student Visa Pathway Student Visa This visa allows you to study up to 3 courses, one after the other, on a single student visa. You must have a Pathway Student Visa offer of place and the funds to pay for your tuition fees. Length of stay Up to 5 years Cost From NZD $375 Processing time 80% with

In [48]:
def build_prompt(query, search_results):
    prompt_template = """
You are an AI assistant specializing in answering questions about New Zealand visas. Your knowledge comes from official New Zealand immigration information. You will be provided with context from relevant articles and a specific question to answer.

First, review the following context:

<context>
{{CONTEXT}}
</context>

Process this context carefully. Each item in the context contains a URL, a header, and main content. Use this information to inform your answers, ensuring you provide accurate and up-to-date information about New Zealand visas.

Now, answer the following question:

<question>
{{QUESTION}}
</question>

To answer the question:
1. Analyze the question and identify the key points related to New Zealand visas.
2. Search through the provided context for relevant information.
3. Formulate a clear, concise answer based on the official information.
4. If the question cannot be fully answered with the given context, state this clearly and provide the most relevant information available.

Write your answer using short markdown syntax, as it will be displayed in a Telegram chat. Use **bold** for emphasis and *italics* for titles or important terms.

Always include at least one relevant URL from the context as a reference. Format the URL reference at the end of your answer like this:
[Source](URL)

If multiple sources are used, include them as separate reference links.

Provide your answer within <answer> tags.
""".strip()

    context = " ".join([json.dumps(x) for x in search_results])
       
    
    prompt = prompt_template.format(QUESTION=query, CONTEXT=context).strip()
    return prompt

def llm(prompt):
    response = clientA.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens = 500,
        messages=[
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": prompt
                }
              ]
            }
      ]
    )
    return response.content[0].text
    

In [49]:
rag(q)

"Certainly! I'll analyze the question, review the provided context, and formulate an answer based on the official New Zealand visa information. I'll follow the guidelines you've outlined for formatting and sourcing.\n\n<answer>\n\n**New Zealand Working Holiday Visa Requirements:**\n\nTo be eligible for a New Zealand Working Holiday Visa, you must:\n\n1. Be aged 18-30 (or 18-35 for some countries)\n2. Have a valid passport\n3. Meet health and character requirements\n4. Have sufficient funds for your stay (typically NZD $4,200)\n5. Have a return ticket or enough money to buy one\n\n*Key points to note:*\n\n- The visa allows you to work and travel in New Zealand for up to 12 months (or 23 months for UK citizens)\n- You must not bring children with you\n- You can study for up to 6 months\n- The application is done online through the Immigration NZ website\n\nIt's important to check the specific requirements for your country, as they may vary slightly.\n\n[Source](https://www.immigration.go